In [19]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.manifold import TSNE
from tqdm import tqdm

from core import ROOT_PATH
from core.embedding_models import FastTextEmbeddingModel
from core.models.clustering_model import ClusteringModel

In [20]:
fast_text_embedding = FastTextEmbeddingModel()

In [21]:
data_path = Path("data/data_vacancies_processed_1k.csv")
df = pd.read_csv(ROOT_PATH / data_path, index_col=0)
df["work_skills"] = df.work_skills.apply(eval)
df.head()

,id,custom_position,schedule,salary_from,salary_to,offer_education_id,education_name,education_is_base,education_order_num,city_id,list_regions,work_skills,tags_id
0,48202096,Сварщик-сборщик,полный рабочий день,60000,120000,0,любое,True,0,2,[4],"[сварочные работы, сборка изделий по чертежам,...",NaN
1,48202097,Сварщик-монтажник,полный рабочий день,60000,120000,0,любое,True,0,2,[4],"[монтажные работы, строительные работы, электр...",NaN
2,48202098,Слесарь-сборщик,полный рабочий день,60000,80000,0,любое,True,0,2,[4],"[работа на фрезерных станках, слесарный ремонт...",NaN
3,48202356,Грузчик-упаковщик,частичная занятость,30000,35000,0,любое,True,0,1,[3],"[комплектация товара, маркировка, стрессоустой...","[6, 9]"
4,48202357,Грузчик-упаковщик,частичная занятость,30000,35000,0,любое,True,0,57,"[181, 182, 183, 185, 186, 187, 188, 189, 190, ...","[маркировка, стрессоустойчивость, погрузочно-р...","[6, 9]"


In [22]:
unique_skills = df.work_skills.explode().unique()
unique_skills_embeddings_fasttext = {
    k: fast_text_embedding.generate(k) for k in tqdm(unique_skills)
}

100%|██████████| 896/896 [00:00<00:00, 112689.93it/s]


In [23]:
df_skills_embeddings = pd.DataFrame(
    list(unique_skills_embeddings_fasttext.items()), columns=["name", "embedding"]
)
df_skills_embeddings.head()

,name,embedding
0,сварочные работы,"[0.042372044, 0.0050952206, 0.008594746, -0.01..."
1,сборка изделий по чертежам,"[0.026281675, 0.05808256, 0.044726472, 0.05175..."
2,ручная дуговая сварка,"[0.033271495, 0.002705688, -0.030051084, 0.085..."
3,электродуговая сварка,"[0.057141438, 0.016982837, -0.020811101, 0.077..."
4,аргонодуговая сварка,"[0.06702779, 0.0045817997, -0.012593956, 0.076..."


In [24]:
model_path = ROOT_PATH / "checkpoints/clustering_model_fasttext.pkl"
clustering_model = ClusteringModel()
clustering_model = clustering_model.load_model(model_path)

Model successfully loaded


In [25]:
df_skills_embeddings["cluster_label"] = df_skills_embeddings.embedding.apply(
    clustering_model.predict
)
df_skills_embeddings.head()

,name,embedding,cluster_label
0,сварочные работы,"[0.042372044, 0.0050952206, 0.008594746, -0.01...",13
1,сборка изделий по чертежам,"[0.026281675, 0.05808256, 0.044726472, 0.05175...",6
2,ручная дуговая сварка,"[0.033271495, 0.002705688, -0.030051084, 0.085...",13
3,электродуговая сварка,"[0.057141438, 0.016982837, -0.020811101, 0.077...",13
4,аргонодуговая сварка,"[0.06702779, 0.0045817997, -0.012593956, 0.076...",13


In [26]:
tsne = TSNE(n_components=2)
embeddings = np.array(df_skills_embeddings["embedding"].tolist())
embeddings_2d = tsne.fit_transform(embeddings)
df_skills_embeddings.loc[:, ["x", "y"]] = embeddings_2d

In [27]:
fig = px.scatter(
    df_skills_embeddings,
    x="x",
    y="y",
    color="cluster_label",
    hover_name="name",
    title="Scatter Plot of Vacancy Embeddings Colored by Cluster Label",
)

fig.update_layout(hovermode="closest")

fig.show()

In [28]:
sample_name = "Программист джанго"
sample_embedding = fast_text_embedding.generate(sample_name)

In [29]:
fig.add_scatter(
    x=[0],
    y=[0],
    marker=dict(color="red", size=10),
    name=sample_name,
)

In [31]:
df_skills_embeddings.to_pickle("demo_clustering_data.pkl")
